# Basics of Mobile Robotics, Project report 

The final project for EPFL's [**MICRO-452: Basics of Mobile Robotics**](https://edu.epfl.ch/coursebook/en/basics-of-mobile-robotics-MICRO-452) course. This project implements a complete navigation stack for a differential drive robot, featuring computer vision, graph-based path planning, sensor fusion (EKF), and reactive control.

**The Concept:** Inspired by the Disney movie WALL·E, the robot EVE must navigate through the interior of the spaceship Axiom - a complex and hazardous environment. Her mission is to reach, her love,  WALL·E as fast as possible, while avoiding a series of dangerous obstacles.
The map includes:

- Windows: Represented as static dark-blue polygonal regions that EVE must not cross. These areas act as boundaries or fatal zones, similar to voids in the spacecraft’s structure.

- Humans (“Passengers”): Large, drifting individuals who move unpredictably through the corridors. They function as dynamic obstacles that EVE must detect and avoid in real time.

![Example environment](../vision_debug/warped.jpg)


## Group members & tasks repartition
- Tancrede Lamort De Gail: Vision, global navigation, motion control
- John Constantin: Vision, global navigation, motion control
- Marcus Edjolo: Local naviation, global navigation, motin control
- Yvan Barragan: Filtering, motion control


## 1. Physical Setup

The project tasks a Thymio II robot with navigating from a start pose to a goal pose while avoiding two distinct types of obstacles.

### 1.1 The Environment

The playground consist of a white rectangle of 130x92 cm delimitated by four Aruco Marker place on each of the 4 corners.

The environment presents a dual-layer challenge:

1.  **Static "Global" Obstacles (The Windows):**
    -   **Physicality:** Flat, dark blue polygonal cutouts.
    -   **Visibility:** _Invisible_ to the robot's onboard horizontal proximity sensors, but clearly visible to the overhead global camera.
    -   **Constraint:** The robot cannot drive over them. Avoiding them relies entirely on accurate global localization and path planning.
2.  **Ephemeral "Local" Obstacles (The Passengers):**
    -   **Physicality:** 3D physical objects (cylinders, blocks) roughly the size of the robot.
    -   **Visibility:** Visible to the robot's onboard sensors, but _ignored_ by the global camera mapping (or placed after mapping is complete).
    -   **Constraint:** These act as dynamic blockers. The robot must use local sensing to reactively avoid them without losing track of its global objective.

### 1.2 The Robot

The **Thymio II** is a differential-drive mobile robot.

-   **Sensors:** 5 front-facing and 2 rear-facing horizontal IR proximity sensors (for local avoidance).
-   **Comms:** Python API interfacing via USB/RF dongle.
-   **Markers:** An ArUco marker is mounted on top of the robot to facilitate high-precision global tracking (Pose: $x, y, \theta$) via the overhead camera.

### 1.3 The Camera

A standard webcam ($1920 \times 1080$, RGB, 30 FPS) is mounted overhead. 



## 2. Vision System
The vision systeme serves as the "GPS" of the system, responsible for:

1.  **Initial Mapping:** Detecting static obstacle polygons and the goal area.
2.  **Live Tracking:** Providing absolute position estimates to correct the robot's drifting odometry.


### 2.1 System Architecture

The vision pipeline operates in two distinct phases:

#### 2.1.1 Initialization Phase (Static Mapping)
During startup, the system captures a single reference frame and extracts the complete static environment:
- Calibration markers define the workspace boundaries
- Static obstacles (the "spacecraft windows") are segmented and converted to polygonal representations
- Goal location (WALL·E's position) is identified
- Visibility graph is pre-computed for path planning

#### 2.1.2 Runtime Phase (Real-Time Localization)
During navigation, the system continuously:
- Captures frames at approximately 10 Hz
- Detects (EVE's position) the robot's ArUco marker
- Computes the robot's pose $(x, y, \theta)$
- Provides measurements to the Extended Kalman Filter for sensor fusion

### 2.2 Coordinate Systems and Perspective Transformation

The vision system must bridge two distinct coordinate frames:

**Camera Frame:** The raw image captured by the overhead camera, measured in pixels with origin at top-left, where coordinates are $(u, v) \in [0, 1920] \times [0, 1080]$.

**World Frame:** The robot's navigation space, measured in centimeters with origin at bottom-left of the workspace, where coordinates are $(x, y) \in [0,132.5 ] \times [0, 92.5]$.

#### 2.2.1 Perspective Correction

Due to the camera's mounting angle and lens distortion, the raw image exhibits perspective distortion. We use a **homography transformation** to warp the image into an orthographic (bird's-eye) view:

$$
\begin{bmatrix} u' \\ v' \\ w \end{bmatrix} = \mathbf{H} \begin{bmatrix} u \\ v \\ 1 \end{bmatrix}
$$

where $\mathbf{H} \in \mathbb{R}^{3 \times 3}$ is the perspective transformation matrix computed via OpenCV's `getPerspectiveTransform()` using the four corner calibration markers as control points.

The final pixel coordinates after perspective correction are:
$$
x_{\text{px}} = \frac{u'}{w}, \quad y_{\text{px}} = \frac{v'}{w}
$$

#### 2.2.2 Metric Conversion

After warping, we convert from the rectified pixel space to world coordinates (cm):

$$
x_{\text{cm}} = \frac{x_{\text{px}}}{\alpha_x}, \quad y_{\text{cm}} = H_{\text{map}} - \frac{y_{\text{px}}}{\alpha_y}
$$

where:
- $\alpha_x = \frac{W_{\text{px}}}{W_{\text{cm}}}$ and $\alpha_y = \frac{H_{\text{px}}}{H_{\text{cm}}}$ are pixel-to-centimeter scaling factors
- $H_{\text{map}}$ is the map height (flipping y-axis from image convention)


### 2.3 Environment Segmentation

#### 2.3.1 Color-Based Segmentation
We segment obstacles and the goal using **HSV color space** thresholding (finding experimentally), which is more robust to lighting variations than RGB:

**Static Obstacles (Dark Blue):**
$$
\text{Mask}_{\text{obstacle}} = \{(h,s,v) \mid 90° \leq h \leq 165°, \, 40 \leq s \leq 255, \, 80 \leq v \leq 255\}
$$

**Goal Region (Red):**
Due to red's wraparound in HSV (0° and 360° are both red), we use a dual-range mask:
$$
\text{Mask}_{\text{goal}} = \text{Mask}_{\text{red1}} \cup \text{Mask}_{\text{red2}}
$$
where:
- $\text{Mask}_{\text{red1}}: 0° \leq h \leq 15°$
- $\text{Mask}_{\text{red2}}: 165° \leq h \leq 180°$


#### 2.3.2 Morphological Filtering
To eliminate noise and fill gaps in the segmented regions, we apply morphological operations with a $7 \times 7$ kernel:
1. **Opening** (erosion → dilation): Removes small noise blobs
2. **Closing** (dilation → erosion): Fills internal holes in obstacles

<div style="display: flex; gap: 20px; justify-content: center; align-items: flex-start;">

  <figure style="margin: 0; text-align: center;">
    <img src="../vision_debug/obstacle_mask.jpg" alt="Obstacle mask" width="300"/>
    <figcaption>Obstacle mask</figcaption>
  </figure>

  <figure style="margin: 0; text-align: center;">
    <img src="../vision_debug/target_mask.jpg" alt="Target mask" width="300"/>
    <figcaption>Target mask</figcaption>
  </figure>
</div>


### 2.4 Configuration Space Construction

#### 2.4.1 Polygon Approximation
Raw contours from segmentation are noisy and contain hundreds of vertices. We simplify them using the **Douglas-Peucker algorithm**:

$$
\epsilon = 0.015 \cdot \text{arcLength}(\text{contour})
$$

This produces polygons with 4-12 vertices while preserving essential shape features.

#### 2.4.2 C-Space Inflation
To account for the robot's physical dimensions, we **buffer** each obstacle polygon by the robot's radius plus a safety margin:

$$
\mathcal{O}_{\text{buffered}} = \mathcal{O} \oplus B_r
$$

where:
- $\mathcal{O}$ is the original obstacle polygon
- $B_r$ is a disk of radius $r = r_{\text{robot}} + \text{padding}$ (≈ 5.5 cm)
- $\oplus$ denotes Minkowski sum (implemented via Shapely's `buffer()` with mitre join style)

This operation effectively reduces the robot to a point in configuration space, simplifying collision checking.

#### 2.4.3 Vertex Merging
The buffering operation can introduce redundant vertices on long straight edges. To reduce graph complexity, we merge vertices closer than a threshold $d_{\text{min}} = 6.0$ cm:

$$
\text{if } \|v_i - v_{i+1}\| < d_{\text{min}}, \quad v_{\text{merged}} = \frac{v_i + v_{i+1}}{2}
$$
### 2.5 ArUco Marker Detection and Pose Estimation

#### 2.5.1 Marker Detection
We use the **4×4 ArUco dictionary** (50 markers) for robust detection. The detection pipeline:
1. Convert warped image to grayscale
2. Apply adaptive thresholding to enhance contrast
3. Detect marker corners using OpenCV's `ArucoDetector`
4. Extract marker ID and corner coordinates

#### 2.5.2 Pose Computation
Given the four corners of the detected marker $\{C_0, C_1, C_2, C_3\}$ (ordered as Top-Left, Top-Right, Bottom-Right, Bottom-Left), we compute:

**Position** (centroid):
$$
x = \frac{1}{4}\sum_{i=0}^{3} C_{i,x}, \quad y = \frac{1}{4}\sum_{i=0}^{3} C_{i,y}
$$

**Orientation** (from marker's left edge):
$$
\theta = \text{atan2}(C_{0,y} - C_{3,y}, \, C_{0,x} - C_{3,x})
$$

This orientation vector points along the robot's forward direction, as the marker is mounted with its left edge aligned with the robot's heading.

### 2.6 Visibility Graph Construction

The visibility graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ encodes all collision-free straight-line paths through the environment.

#### 2.6.1 Node Set
The vertices include:
$$
\mathcal{V} = \mathcal{V}_{\text{obstacles}} \cup \{v_{\text{goal}}\} \cup \{v_{\text{start}}\}
$$

where $\mathcal{V}_{\text{obstacles}}$ contains all vertices of all buffered obstacle polygons.

#### 2.6.2 Edge Set
Two nodes $v_i, v_j$ are connected by an edge if:
1. The line segment $\overline{v_i v_j}$ does not intersect any obstacle interior
2. Neither endpoint is inside an obstacle

We implement visibility checking using Shapely's geometric predicates:

**Visibility Test:**
$$
\text{visible}(v_i, v_j) = \begin{cases}
\text{True} & \text{if } \overline{v_i v_j} \cap \mathcal{O}_k = \emptyset \, \forall k \\
\text{False} & \text{otherwise}
\end{cases}
$$

**Edge Weight:**
$$
w(v_i, v_j) = \|v_i - v_j\|_2 = \sqrt{(x_j - x_i)^2 + (y_j - y_i)^2}
$$

#### 2.6.3 Shrinking Heuristic
To avoid false positives from floating-point precision issues at polygon vertices, we test a **slightly shortened** version of each line segment:

$$
\overline{v_i v_j}_{\text{test}} = \{v_i + 0.001 \cdot (v_j - v_i) \to v_i + 0.999 \cdot (v_j - v_i)\}
$$

This prevents spurious intersections when paths graze obstacle corners.

### 2.7 Dynamic Graph Augmentation

Since the robot's start position changes after each kidnapping event, the visibility graph must be **dynamically updated**:

1. **Base Graph:** Contains only obstacle vertices and the goal (precomputed once)
2. **Runtime Augmentation:** When the robot is relocated:
   - Add a new start node $v_{\text{start}}$ at the robot's current pose
   - Connect $v_{\text{start}}$ to all visible nodes in the base graph
   - Run A* search on the augmented graph

This avoids recomputing the entire $O(n^3)$ visibility graph (where $n \approx 40$ nodes) after every kidnapping.

### 2.8 Measurement Uncertainty

The camera provides pose estimates $(x, y, \theta)$ with empirically measured covariance:

$$
\mathbf{R}_{\text{camera}} = \begin{bmatrix}
0.00119 & 0 & 0 \\
0 & 0.00179 & 0 \\
0 & 0 & 0.000061
\end{bmatrix}
$$

This translates to standard deviations of approximately:
- Position: $\sigma_x \approx 3.4$ mm, $\sigma_y \approx 4.2$ mm
- Orientation: $\sigma_\theta \approx 0.45°$

These values were obtained through repeated measurements of a stationary robot and used to tune the Extended Kalman Filter (detailed in Section X).

---

**Key Design Decisions:**
- **Color segmentation over feature detection:** Reliable under controlled lighting, faster than edge detection
- **ArUco markers over natural features:** Sub-pixel accuracy, computationally cheap, robust to partial occlusion
- **Visibility graph over grid-based methods:** Produces optimal paths for polygonal environments, fewer nodes than probabilistic roadmaps
- **Single mapping phase:** Assumes static obstacles (dynamic obstacles handled by local reactive layer)
 

## Global navigation
Detailing of global navigation (_move_to_point function), A* implemenation, node creations etc


## ⚙️ Motion Control Module Implementation

The robot's motion control is managed by the ThymioController class, which executes a Finite State Machine (FSM) to switch seamlessly between global, goal-oriented driving and local, reactive avoidance.

---

### 1. The FSM and Blended Control Design

#### Theory: FSM with Time-Based Control Blending

The FSM design is a classic solution for managing mutually exclusive robot behaviors (NAVIGATING vs. AVOIDING).

A key design choice replaces traditional instantaneous state switching with a time-based control blending mechanism when exiting the AVOIDING state. This technique addresses a critical issue in robotics: control signal instability upon re-entry to complex goal-seeking behavior.

The transition from avoidance back to navigation is governed by a fixed-duration cooldown phase (AVOIDANCE_DURATION = 4.0 s). During this time, the output motor commands are a linear interpolation (or blending) between the simplest stable command (Drive Straight) and the complex command (Move to Point).

The blending factor, $\text{decay}$, ensures a smooth ramp-down of the straight-drive influence:
$$\text{Command} = \text{decay} \cdot \text{Drive}_{\text{Straight}} + (1.0 - \text{decay}) \cdot \text{Command}_{\text{Navigate}}$$
where $\text{decay} \to 1.0$ at the start of the cooldown and $\text{decay} \to 0.0$ at the end.

#### Choices and Justification

* Fixed Cooldown Duration ($\mathbf{4.0\text{ s}}$): This fixed time, rather than a proportional distance or angular correction, was chosen for its robustness in the constrained Thymio environment. It guarantees a minimum time for the robot's pose estimator (not shown in the controller code) to settle and for the robot to clear a reasonable distance from the disturbance.
* Avoidance Threshold ($\mathbf{400}$): This low $\text{SENSOR\_THRESHOLD}$ was selected to trigger avoidance preemptively (at a greater distance) given the robot's high $\text{AVOID\_SPEED}$ ($\mathbf{125.0}$). Early reaction is preferred over late, aggressive braking.

---

### 2. Goal-Oriented Driving: $\text{\_move\_to\_point()}$

This function computes the necessary wheel speeds to steer the robot toward the current waypoint, forming the core of the NAVIGATING state.

#### Theory: P-Controller with Regulated Linear Speed

The controller uses a Proportional (P) Controller for angular control, coupled with a regulated linear speed function based on the $\cos$ of the heading error.

* Angular Control ($\omega$): $\omega$ is proportional to the $\text{heading\_error}$.
* Linear Speed ($V$): $V$ is attenuated by $\max(0.0, \cos(\text{heading\_error}))$. This design ensures the robot slows down when it needs to make large turning corrections (i.e., when $\text{heading\_error}$ approaches $\pm 90^\circ$), resulting in a stable, efficient turn-in-place maneuver instead of a large arcing path.

#### Parameter Tuning

| Parameter | Value | Rationale for Choice |
| :--- | :--- | :--- |
| $\text{KP\_ROT}$ | $\mathbf{120.0}$ | A high proportional gain, empirically tuned for quick, decisive steering with minimal oscillation around the target heading. |
| $\text{MAX\_SPEED}$ | $\mathbf{125.0}$ | Set high to ensure high path efficiency when driving straight, matching the $\text{AVOID\_SPEED}$. |

---

### 3. Obstacle Avoidnce: $\text{\_avoid\_obstacles()}$

This function generates the motor commands during the AVOIDING state, taking over path following to prevent collision with a local obstacle.

#### Theory: Differential Steering with Dynamic Weighting

The avoidance mechanism is a differential steering technique where the $\text{turn}_{\text{weight}}$ is derived from the weighted difference between the left and right sensor groups.

A key design innovation is the use of dynamic sensor weighting based on the symmetry of the frontal threat:

$$\text{turn}_{\text{weight}} = (S_0 \cdot W_{\text{outer}} + S_1 \cdot W_{\text{inner}}) - (S_3 \cdot W_{\text{inner}} + S_4 \cdot W_{\text{outer}})$$

If the inner sensors ($\text{sensor}[1]$ and $\text{sensor}[3]$) are roughly equal ($\text{symmetric threat}$), the controller assumes a head-on situation and increases the weight of the outer sensors ($W_{\text{outer}}$) while setting $W_{\text{inner}} = 0$. This forces the robot to ignore the central sensors and commit quickly to the side with the slightest opening. Otherwise, the base weights ($\mathbf{1.7}$ inner, $\mathbf{1.0}$ outer) are used for finer correctional steering.

#### Parameter Tuning

| Parameter | Value | Rationale for Choice |
| :--- | :--- | :--- |
| $\text{AVOID\_SPEED}$ | $\mathbf{125.0}$ | The avoidance speed is intentionally set equal to $\text{MAX\_SPEED}$. This decision prioritizes clearing the obstacle area quickly, relying on the aggressive $\text{angular\_speed}$ calculation to manage the turn rather than slowing down. |
| $\text{Rotation Gain}$ | $\mathbf{0.025}$ | A low gain applied to the $\text{turn}_{\text{weight}}$. This is necessary because the $\text{turn}_{\text{weight}}$ can be a very large number (sum of sensor readings). The low gain scales the powerful sensor input into a stable, non-spinning rotational impulse. |
## Code

In [ ]:
def update(self, current_pose: Pose, target_pos: Point, sensor_data: list[int]) -> tuple[float, float]:
        """
        Update control commands based on current pose, target position, and sensor data.
        """
        current_time = time.time()

        # Check for obstacles in front 5 sensors
        max_front_sensor = max(sensor_data[0:5])
        is_obstacle_present = max_front_sensor > self.SENSOR_THRESHOLD

        # Calculate Navigation Component (Base behavior)
        nav_l, nav_r = self._move_to_point(current_pose, target_pos)

        if is_obstacle_present:
            self.state = RobotState.AVOIDING
            self.last_avoidance_time = current_time
            # Active avoidance: use sensors directly
            return self._avoid_obstacles(sensor_data)

        elif self.state == RobotState.AVOIDING:
            elapsed = current_time - self.last_avoidance_time
            if elapsed < self.AVOIDANCE_DURATION:
                # Decay phase: Blend "Drive Straight" with "Move to Point"
                # 1.0 = Drive Straight, 0.0 = Move to Point
                decay = max(0.0, 1.0 - (elapsed / self.AVOIDANCE_DURATION))

                # Drive straight component
                straight_l = self.MAX_SPEED
                straight_r = self.MAX_SPEED

                # Blend controls
                l_cmd = decay * straight_l + (1.0 - decay) * nav_l
                r_cmd = decay * straight_r + (1.0 - decay) * nav_r

                return int(l_cmd), int(r_cmd)
            else:
                self.state = RobotState.NAVIGATING

        # Default: Navigation
        return nav_l, nav_r

## 🧭 Local Navigation: Reactive Obstacle Avoidance

This module provides an **immediate, priority control signal** whenever an obstacle is detected, overriding the goal-seeking behavior of the navigation system. The avoidance mechanism is split into an **Active Evasion Phase** and a **Blended Cooldown Phase**.

---

### Theory: Differential Steering and Control Blending

The system transitions through two distinct control modes:

1.  Avoidance: When an obstacle is detected ($\text{sensor} > \mathbf{400}$), the avoidance mode is triggered and the robot calculates a rotational impulse based on the obstacle distance from the triggered sensors. The goal is to induce a rapid pivot away from the side experiencing the sensor reading.

2.  Blended Cooldown: Once the obstacle clears, the robot enters a time-based decay phase ($\mathbf{4.0\text{ s}}$ duration). During this time, the commanded wheel speeds are a linear blend between a maximum straight-line drive ($\text{MAX\_SPEED}$) and the navigation commands ($\text{nav}_{\text{l}}, \text{nav}_{\text{r}}$). This blending prevents sterring back on the obstacle and a sudden oversteering back towards the target to ensures a smooth, controlled re-entry to the navigation path.

#### Avoidance Drive Calculation

The wheel speed calculation during the avoidance mode directly manipulates the motor targets to achieve the differential effect:

$$
L_{\text{speed}} = \text{AVOID}_{\text{SPEED}} + \text{turn}_{\text{weight}} \times \text{gain}
$$
$$
R_{\text{speed}} = \text{AVOID}_{\text{SPEED}} - \text{turn}_{\text{weight}} \times \text{gain}
$$

A positive $\text{turn}_{\text{weight}}$ (obstacle on the left) accelerates the left wheel and decelerates the right wheel, forcing a turn right, and the opposite happens for a negative $\text{turn}_{\text{weight}}$.

### Implementation Details

* Turning Weight Calculation: The $\text{turn}_{\text{weight}}$ is calculated using dynamic sensor weighting. Base weights ($\mathbf{1.7}$ inner, $\mathbf{1.0}$ outer) are applied unless the obstacle is frontal/symmetric ($\text{sensor}[1] \approx \text{sensor}[3]$), in which case inner sensors are ignored ($\mathbf{0.0}$) and outer sensors are heavily prioritized ($\mathbf{2.3}$) to force a sharp directional evasion.

    $$
    \text{turn}_{\text{weight}} = (S_0 \cdot W_{\text{outer}} + S_1 \cdot W_{\text{inner}}) - (S_3 \cdot W_{\text{inner}} + S_4 \cdot W_{\text{outer}})
    $$

* Blend Logic: After avoidance, the control switches to blending for $\mathbf{4.0\text{ s}}$. The decay factor ($\text{decay}$) scales the mix:

    $$\text{Command} = \text{decay} \cdot \text{Straight} + (1.0 - \text{decay}) \cdot \text{Navigate}$$

    This ensures the robot gradually shifts from prioritizing forward stability back to goal-seeking control.

# Code:

In [ ]:
def _avoid_obstacles(self, sensor_data: list[int]) -> tuple[float, float]:
        """
        Simple obstacle avoidance behavior based on proximity sensor readings.
        """

        # Simple Braitenberg-style logic:
        # "If left sensor sees something, speed up right wheel (turn right)"
        # "If right sensor sees something, speed up left wheel (turn left)"

        # Map sensors to weights (simplified example)
        # sensors: [LeftMost, Left, Center, Right, RightMost]

        # Calculate a "turning force" based on sensor difference
        # Positive = Turn Right, Negative = Turn Left
        inner_sensor_weight = 1.7
        outer_sensor_weight = 1.0
        if abs(sensor_data[1] - sensor_data[3]) < self.FRONT_SENSOR_MIN_DELTA:
            inner_sensor_weight = 0.0
            outer_sensor_weight = 2.3

        turn_weight = (
            sensor_data[0] * outer_sensor_weight
            + sensor_data[1] * inner_sensor_weight
            - sensor_data[3] * inner_sensor_weight
            - sensor_data[4] * outer_sensor_weight
        )

        # Apply gain to turn force
        angular_speed = turn_weight * 0.025

        l_speed = self.AVOID_SPEED + angular_speed
        r_speed = self.AVOID_SPEED - angular_speed

        return int(l_speed), int(r_speed)

# Referances and inspirations